In [1]:
from argparse import Namespace


In [2]:
configs = Namespace(
    is_training=1, 
    model_id='ETTm2_96_96', 
    model='Autoformer', 
    data='ETTm2', 
    root_path='./dataset/ETT-small/', 
    data_path='ETTm2.csv', 
    features='S', 
    target='OT', 
    freq='t', 
    checkpoints='./checkpoints/', 
    seq_len=96, 
    label_len=96, 
    pred_len=96, 
    bucket_size=4, 
    n_hashes=4, 
    enc_in=1, 
    dec_in=1, 
    c_out=1, 
    d_model=512, 
    n_heads=8, 
    e_layers=2, 
    d_layers=1, 
    d_ff=2048, 
    moving_avg=25, 
    factor=3, 
    distil=True, 
    dropout=0.05, 
    embed='timeF', 
    activation='gelu', 
    output_attention=False, 
    do_predict=False, 
    num_workers=10, 
    itr=1, 
    train_epochs=10, 
    batch_size=32, 
    patience=3, 
    learning_rate=0.0001, 
    des='Exp', 
    loss='mse', 
    lradj='type1', 
    use_amp=False, 
    use_gpu=True, 
    gpu=0, 
    use_multi_gpu=False, 
    devices='0,1,2,3'
)

In [3]:
from models import Autoformer

In [4]:
from exp.exp_main import Exp_Main

In [5]:
Exp = Exp_Main

In [6]:
af = Autoformer.Model(configs)

In [7]:
model_path = "checkpoints/ETTm2_96_96_Autoformer_ETTm2_ftS_sl96_ll96_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0/checkpoint.pth"

In [8]:
import torch
af.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
af.eval()

Model(
  (decomp): series_decomp(
    (moving_avg): moving_avg(
      (avg): AvgPool1d(kernel_size=(25,), stride=(1,), padding=(0,))
    )
  )
  (enc_embedding): DataEmbedding_wo_pos(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=5, out_features=512, bias=False)
    )
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (dec_embedding): DataEmbedding_wo_pos(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=5, out_features=512, bias=False)
    )
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (encoder): Encode

In [10]:
from data_provider import data_factory

In [11]:
data_set, data_loader = data_factory.data_provider(configs, 'test')

checking the data: <class 'data_provider.data_loader.Dataset_ETT_minute'>
called Dataset_ETT_minute init
checking size: [96, 96, 96]
len(self.data_x): 586
self.seq_len: 96
self.pred_len: 96
to return data: =395
test 395


In [12]:
data_loader

In [13]:
data_set[0][0].shape

(96, 1)

In [16]:
len(data_set[0])

4

In [18]:
device = torch.device("cpu")

In [21]:
os = []
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)
    
    batch_x_mark = batch_x_mark.float().to(device)
    batch_y_mark = batch_y_mark.float().to(device)

    dec_inp = torch.zeros_like(batch_y[:, -configs.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
        
    # model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    output = af(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    os.append(output)

len(self.data_x): 586
self.seq_len: 96
self.pred_len: 96
to return data: =395


In [23]:
len(os)

13

In [25]:
os[0].shape

torch.Size([32, 96, 1])

In [26]:
data_set, data_loader = data_factory.data_provider(configs, 'train')

checking the data: <class 'data_provider.data_loader.Dataset_ETT_minute'>
called Dataset_ETT_minute init
checking size: [96, 96, 96]
len(self.data_x): 9000
self.seq_len: 96
self.pred_len: 96
to return data: =8809
train 8809
len(self.data_x): 9000
self.seq_len: 96
self.pred_len: 96
to return data: =8809
len(self.data_x): 9000
self.seq_len: 96
self.pred_len: 96
to return data: =8809


In [28]:
data_set[0]

(array([[-0.07926291],
        [-0.13784069],
        [-0.25512962],
        [-0.28448545],
        [-0.28448545],
        [-0.31384128],
        [-0.31384128],
        [-0.37241906],
        [-0.37241906],
        [-0.37241906],
        [-0.46035268],
        [-0.57764212],
        [-0.75357552],
        [-0.84157581],
        [-0.9001541 ],
        [-0.92950942],
        [-0.98815414],
        [-1.01744304],
        [-1.04679886],
        [-1.10537665],
        [-1.16408805],
        [-1.16408805],
        [-1.13473222],
        [-1.19337694],
        [-1.22266583],
        [-1.25202166],
        [-1.25202166],
        [-1.25202166],
        [-1.25202166],
        [-1.25202166],
        [-1.25202166],
        [-1.25202166],
        [-1.28137724],
        [-1.31066613],
        [-1.31066613],
        [-1.31066613],
        [-1.34002196],
        [-1.48653335],
        [-1.5452445 ],
        [-1.5452445 ],
        [-1.57453365],
        [-1.57453365],
        [-1.57453365],
        [-1